In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/DataSet/cleaned.csv')

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy.shape

(6071, 3)

In [ ]:
df_copy.head()

,questions,Answers,Unnamed: 2
0,How is the write?,excellent writing,Books
1,What impact did the book have?,It 's just a shame that what the book had to o...,Books
2,Is the plot line good enough?,The book got me hooked almost immediately Char...,Books
3,Which is the good premise principal?,premise is interesting,Books
4,How is the book?,The escapee turns out to be Sirius Black the,Books


In [ ]:
df_copy = df_copy.rename(columns = {"Unnamed: 2" : "Y"})

In [ ]:
df_copy.head()

,questions,Answers,Y
0,How is the write?,excellent writing,Books
1,What impact did the book have?,It 's just a shame that what the book had to o...,Books
2,Is the plot line good enough?,The book got me hooked almost immediately Char...,Books
3,Which is the good premise principal?,premise is interesting,Books
4,How is the book?,The escapee turns out to be Sirius Black the,Books


In [ ]:
df_copy['questions'] = df_copy['questions'].str.lower()
df_copy['questions'] = df_copy['questions'].str.replace("?", "")
df_copy['tokanized'] = df_copy['questions'].apply(lambda x: x.split(' '))
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
df_copy['tokanized'] = df_copy['tokanized'].apply(lambda x: [item for item in x if item not in stop_words])


In [ ]:
df_copy.head()

,questions,Answers,Y,tokanized
0,how is the write,excellent writing,Books,[write]
1,what impact did the book have,It 's just a shame that what the book had to o...,Books,"[impact, book]"
2,is the plot line good enough,The book got me hooked almost immediately Char...,Books,"[plot, line, good, enough]"
3,which is the good premise principal,premise is interesting,Books,"[good, premise, principal]"
4,how is the book,The escapee turns out to be Sirius Black the,Books,[book]


In [ ]:
df_copy.isnull().sum()

questions    0
Answers      0
Y            0
tokanized    0
dtype: int64

In [ ]:
# data_questions = df_copy['tokanized'].apply(lambda x: [x]).tolist() 
data_questions = np.asarray(df_copy['tokanized'])
data_questions

array([list(['write']), list(['impact', 'book']),
       list(['plot', 'line', 'good', 'enough']), ..., list(['hotel']),
       list(['parking']), list(['stay'])], dtype=object)

In [ ]:
df_copy['tokanized'].dtypes

dtype('O')

In [ ]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
data_for_training_questions = list(tagged_document(data_questions))
print(data_for_training_questions[:1])

[TaggedDocument(words=['write'], tags=[0])]


In [ ]:
model_questions = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=1, epochs=30)
model_questions.build_vocab(data_for_training_questions)
model_questions.train(data_for_training_questions, total_examples=model_questions.corpus_count, epochs=model_questions.epochs)

In [ ]:
# data_answers = df_copy.Answers.values.tolist()
data_answers = df_copy['Answers'].apply(lambda x: [x]).tolist() 
data_answers = np.asarray(data_answers)
data_answers

array([['excellent writing'],
       ["It 's just a shame that what the book had to offer was no surprise to me"],
       ['The book got me hooked almost immediately Characters and dialogue are good but I liked the movie better'],
       ...,
       ['this upscale hotel is highly recommended and its location is hard to beat'],
       ['The parking rate is ridiculously high'],
       ['The Hotel Adagio is absolutely FABULOUS']], dtype='<U429')

In [ ]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
data_for_training_answers = list(tagged_document(data_answers))
print(data_for_training_answers[:1])

[TaggedDocument(words=array(['excellent writing'], dtype='<U429'), tags=[0])]


In [ ]:
model_answers = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)
model_answers.build_vocab(data_for_training_answers)
model_answers.train(data_for_training_answers, total_examples=model_answers.corpus_count, epochs=model_answers.epochs)

In [ ]:
answer1 = model_answers.infer_vector(['excellent', 'writing'])
answer1

array([-0.00358992,  0.01163803,  0.00492914,  0.00492813,  0.00619977,
       -0.0047428 , -0.01113535,  0.00769555, -0.00433497, -0.00966512,
       -0.00595968, -0.00556116, -0.00633477, -0.00981487, -0.00616774,
       -0.01177523, -0.01175572,  0.00687055,  0.00611734, -0.00607264,
        0.01188127, -0.0020296 ,  0.00157827, -0.00193782, -0.01222141,
       -0.00309405, -0.00805303,  0.01073469, -0.00129026,  0.01259939,
        0.00621524,  0.00783388, -0.00874103,  0.00928554, -0.00185307,
       -0.00817985, -0.0063961 ,  0.00383142,  0.01172819,  0.00499512],
      dtype=float32)

In [ ]:
df_copy['tokanized']

0                                [write]
1                         [impact, book]
2             [plot, line, good, enough]
3             [good, premise, principal]
4                                 [book]
                      ...               
6066    [provide, variety, fresh, fruit]
6067          [company, great, security]
6068                             [hotel]
6069                           [parking]
6070                              [stay]
Name: tokanized, Length: 6071, dtype: object

In [ ]:
text_questions = []
text1 =""
for i in range(0, (len(df_copy))):
  text1 = model_questions.infer_vector(df_copy['tokanized'][i])
  text_questions.append(text1)

In [ ]:
text_questions[0]

array([ 0.11424679, -0.05039364, -0.02274443, -0.11224426, -0.02521368,
        0.02848174,  0.07204197, -0.05895597,  0.12337315, -0.05156697,
       -0.03156791,  0.0104045 ,  0.04283414,  0.08917993,  0.02597294,
       -0.05183435,  0.07645292,  0.01934693, -0.0039937 , -0.00864717,
       -0.01387371,  0.02897566, -0.05878798, -0.12757567,  0.01895954,
       -0.0188218 ,  0.06629013,  0.07236961,  0.11753877, -0.00521357,
        0.06813588,  0.01992505, -0.03972434, -0.01539173, -0.08297485,
        0.02261083,  0.01194965, -0.05576629, -0.02114967,  0.02189767],
      dtype=float32)

In [ ]:
data_y = np.asarray(df_copy.Y)
data_y

array(['Books', 'Books', 'Books', ..., 'tripadvisor', 'tripadvisor',
       'tripadvisor'], dtype=object)

In [ ]:
text_questions = np.stack([i.tolist() for i in text_questions])

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df_copy.Y)
y_train = encoder.transform(df_copy.Y)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
model_predict_questions = Sequential()              
model_predict_questions.add(Dense(128, input_dim=text_questions.shape[1], activation= 'relu' ))
model_predict_questions.add(Dropout(0.4))
model_predict_questions.add(Dense(64, activation= 'tanh' ))
model_predict_questions.add(Dropout(0.4))
model_predict_questions.add(Dense(6, activation='softmax'))
model_predict_questions.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model_predict_questions.summary()
model_predict_questions.fit(text_questions, y_train, epochs=500, verbose=1)
scores = model_predict_questions.evaluate(text_questions, y_train)
print (scores)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5248      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
190/190 [==============================] - 15s 2ms/step - loss: 1.7837 - accuracy: 0.2236
Epoch 2/500
190/

In [ ]:
Question_Prob_Score = model_predict_questions.predict([[-5.6933826e-03, -7.1266070e-03, -6.0732388e-03, -3.7099647e-03,
       -4.3845186e-03, -4.8060194e-03,  5.8907567e-04,  8.9565301e-03,
       -2.7781818e-04,  8.1788274e-03, -1.1355451e-02,  1.2013778e-02,
       -1.1618367e-02, -4.0041684e-04,  8.1981951e-03,  7.9985354e-03,
       -9.8783765e-03, -1.2484683e-02,  1.6683192e-03,  8.3702430e-03,
       -1.1477277e-02,  1.0705156e-02,  1.1443961e-02, -5.9133214e-03,
        3.7552789e-03,  5.4123350e-03,  7.5901686e-03, -3.2808478e-03,
       -8.0178697e-06,  1.1579112e-02, -1.1187740e-02, -5.4712980e-03,
        6.8849633e-03,  1.5279922e-03, -8.7181134e-03,  9.3283039e-03,
       -1.1335022e-02,  7.3210071e-03,  1.2493558e-04, -2.3488400e-03]])

In [ ]:
Question_Prob_Score

array([[0.00454157, 0.06136133, 0.05578798, 0.7294206 , 0.1073968 ,
        0.04149183]], dtype=float32)

In [ ]:
Question_Prob_Score = Question_Prob_Score.tolist()

In [ ]:
max_index = []
max_value = max(Question_Prob_Score)
max_index = Question_Prob_Score.index(max_value)
max_index

0

In [ ]:
encoder.inverse_transform([max_index])

array(['Books'], dtype=object)

In [ ]:
# from scipy import spatial
# cos_distance = spatial.distance.cosine(question1, answer1)
# cos_distance
model_questions.wv.vectors.shape

(1491, 40)

In [ ]:
words=list(model_questions.wv.vocab)
print(words)

['write', 'impact', 'book', 'plot', 'line', 'good', 'enough', 'premise', 'principal', 'concept', 'think', 'sibling', 'relationship', 'consider', 'author', 'person', 'skilled', 'describe', 'flow', 'exciting', 'part', 'story', 'people', 'angry', 'release', 'novel', 'martin', 'central', 'idea', '\u200b\u200bthe', 'free', 'imagination', 'feel', 'lot', 'chest', 'pain', '', 'readable', 'pace', 'series', 'feeling', 'one', 'threatening', 'villains', 'parts', 'like', 'kind', 'storyline', 'contain', 'pleasant', 'research', "it's", 'interesting', 'back', 'dialogue', 'sex', 'scene', 'way', 'adventure', 'message', 'reader', 'friend', 'based', 'rumor', 'real', 'opinion', 'give', 'us', 'istory', 'evil', 'many', 'chapters', 'missing', 'funny', 'contains', 'action', 'end', 'depressing', 'stuff', 'insight', 'gift', 'else', 'get', 'emotion', 'began', 'hot', 'hide', 'secret', 'bugs', 'registered', 'valid', 'point', 'probably', 'complicated', 'details', 'favorite', 'quality', 'style', 'world', 'building', 

In [ ]:
model_answers.wv.vectors.shape

(446, 40)

In [ ]:
words=list(model_answers.wv.vocab)
print(words)

['The writing is excellent', 'this story is simply fantastic', 'Everything about the story was interesting and educational', 'The research necessary to write this book is impressive', 'beautiful story', 'good', 'this book tore a hole through my heart', 'The book was a real reflection on the goodness in some people and the evil in others', 'amazing', 'The story is a mystery novel', 'It is an excellent read', 'I loved this book', 'this one is just as good', 'I loved it', 'This series was great', 'This book was terrible', 'the sex is surprisingly bland', 'This book is absurdly bad', 'An absolutely fantastic book', 'The research to write this book was amazing', 'This series is wonderful', 'EXCELLENT', 'The plot is very good', 'This is a very good book', 'The premise was interesting', 'This was a really good story', 'incredible', 'great chemistry', 'the end was stupid', 'the characters are well drawn', 'The writing is horrible', 'This book was really good and surprisingly deep', 'It was so 

In [ ]:
model_answers.most_similar(['small'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('delightful and incredibly helpful staff', 0.44622084498405457),
 ('I loved it', 0.4434046447277069),
 ('This', 0.4351232647895813),
 ('This camera is fantastic', 0.40405070781707764),
 ('Parking is very convenient in their own garage', 0.40143734216690063),
 ('Good taste', 0.37378042936325073),
 ('The decor was fun though', 0.3488338589668274),
 ('GREAT portions and prices', 0.34237539768218994),
 ('They are just wonderful', 0.32423657178878784),
 ('The lobby is Great', 0.30817711353302)]